[132.0, 13.25, 0, 6.87, 0]
[528.0, 13.25, 0, 6.87, 0]


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint, ode
 
"""
Reddyhoff et al. (2015)
https://www.sciencedirect.com/science/article/pii/S0022519315004142
"""
 
def dFdt_original(F,t):
    """
    System of differential equations.
    Equations 1-5 in Reddyhoff et al. (2015)

    Notation
    ________
    P: Paracetamol (APAP)
    S: Sulphate (PAPS)
    N: NAPQI
    G: GSH
    C: Protein adducts

    """
    dF = [0,0,0,0,0] # P, S, N, G, C
    P = F[0]
    S = F[1]
    N = F[2]
    G = F[3]
    C = F[4]
 
    dF[0] = -k_s*S*P - k_G*P - k_450*P + k_N*N
    dF[1] = -k_s*S*P + b_s - d_s*S
    dF[2] = k_450*P - k_N*N - k_GSH*N*G - k_PSH*N
    dF[3] = -k_GSH*N*G + b_G - d_G*G
    dF[4] = k_PSH*N
   
    return dF

### Model parameters ###

# Note: 1 femtomole (fmole) = 10^-15 mol
d_G = 2 # 1/day
b_G = (1.374e-14) * 10**15 # femtomoles/cell/day
k_GSH = (1.6e18) * 10**-15 # cell/femtomoles/day
k_G = 2.99 # 1/day
k_s = (2.26e14) * 10**-15 # cell/femtomoles/day
b_s = (2.65e-14) * 10**15 # femtomoles/cell/day
d_s = 2 # 1/day
k_450 = 0.315 # 1/day
k_N = 0.0315 # 1/day
k_PSH = 110 # 1/day

### Setup for the simulation ###

# titles for plots:
variables = ["Paracetamol (P)", "Sulphation (S)", "NAPQI (N)", "GSH (G)", "Conjugates (C)"]
# the 2 initial conditions for P0 that the paper used:
P0 = [(1.32e-13) * 10**15, (5.28e-13) * 10**15] # femtomoles/cell

# times
t_min = 0
t_max = 5
dt = 0.001
times = np.arange(t_min, t_max+dt, dt) #generate time-grid list

### Running the simulation and plotting ###

fig, axes = plt.subplots(5, 2, figsize=(8, 12))

for i, initial_P0 in enumerate(P0):
    # Running a simulation for each case of P0:
    F0 = [initial_P0, b_s/d_s, 0, b_G/d_G, 0]
    print(F0)
    F = odeint(dFdt_original, F0, times)  # Run simulation
    for j, variable in enumerate(variables):
        # Plotting each subplot:
        axes[j, i].plot(times, F[:,j] * 10**-15, color='midnightblue')
        axes[j, i].set_xlabel("Time (Days)")
        axes[j, i].set_ylabel(f"{variables[j]} (mole)")
        axes[j, i].set_title(f"$P_0 = {initial_P0}$ mole/cell")
        axes[j, i].set_xlim(0, 5)
        axes[j, i].set_ylim(0,)

plt.tight_layout()
plt.show()

[0, 4, 8, 12, 24, 28, 32, 36, 48, 52, 56, 60, 72, 76, 80, 84, 96, 100, 104, 108, 120]
[33.0, 13.25, 0, 6.87, 0]
[132.0, 13.25, 0, 6.87, 0]


In [5]:
### Setting up times (in hours) to take acetaminophen ###

P0 = [((1.32e-13) * 10**15)/4, ((5.28e-13) * 10**15)/4] # femtomoles/cell

# making list like dose_times = [0, 4, 8, 12, 24, 28, 32, 36, 48 ...]
dose_times = []
for day in range(5):
    dose_times.append(24*day)
    for n in range(3):
        dose_times.append(24*day + 4*(n+1))
dose_times.append(24*5)
print(dose_times)

fig, axes = plt.subplots(5, 2, figsize=(8, 12))

for i, initial_P0 in enumerate(P0):
    # Running a simulation for each case of P0:
    F0 = [initial_P0, b_s/d_s, 0, b_G/d_G, 0]
    print(F0)
    for t in range(len(dose_times)-1):
        # setting up a 4 or 12 hour time period, depending on the time of day
        t_min = 0
        t_max = (dose_times[t+1] - dose_times[t])/24
        times = np.arange(t_min, t_max+dt, dt)
        F = odeint(dFdt_original, F0, times)  # Run simulation
        #print(F.shape)
        for j, variable in enumerate(variables):
            # Plotting each subplot:
            # the times are offset by the dose time
            axes[j, i].plot(times + dose_times[t]/24, F[:,j] * 10**-15, color='midnightblue')
            axes[j, i].set_xlabel("Time (Days)")
            axes[j, i].set_ylabel(f"{variables[j]} (mole)")
            axes[j, i].set_title(f"$P_0 = {initial_P0}$ mole/cell")
            axes[j, i].set_xlim(0, 5)
            #axes[j, i].set_ylim(0,)
        # the final values become the initial conditions for the next dose:
        F0 = [F[-1,0]+P0[i], F[-1,1], F[-1,2], F[-1,3], F[-1,4]]
        #print([F[-1,0]+P0[i], F[-1,1], F[-1,2], F[-1,3], F[-1,4]])

plt.tight_layout()
plt.show()

Note:

- I added 132 fmoles/cell or 528 fmoles/cell at each iteration in the plots above -- this parameter might be wrong. It's in the line:

`F0 = [F[-1,0]+P0[i], F[-1,1], F[-1,2], F[-1,3], F[-1,4]]`

- Also, NAPQI looks off.